In [ ]:
import os
import shutil
import tempfile
import matplotlib.pyplot as plt
import PIL
import torch
import numpy as np
from sklearn.metrics import classification_report

from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import decollate_batch, DataLoader
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
from monai.transforms import (
    Activations,
    EnsureChannelFirst,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
)
from monai.utils import set_determinism

print_config()

In [ ]:

root_dir = os.environ.get("/home/yfang/MRI_95_result")#存模型权重和结果的文件夹
data_dir='dataset/MRI_95_merge'#存数据的文件夹

In [ ]:
#设定随机种子
set_determinism(seed=0)

In [ ]:
#对图片的数量，图片的大小，标签，图片的数量进行可视化
class_names = sorted(x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, x)))
num_class = len(class_names)
image_files = [
    [os.path.join(data_dir, class_names[i], x) for x in os.listdir(os.path.join(data_dir, class_names[i]))]
    for i in range(num_class)
]
num_each = [len(image_files[i]) for i in range(num_class)]
image_files_list = []
image_class = []
for i in range(num_class):
    image_files_list.extend(image_files[i])
    image_class.extend([i] * num_each[i])
num_total = len(image_class)
image_width, image_height = PIL.Image.open(image_files_list[0]).size

print(f"Total image count: {num_total}")
print(f"Image dimensions: {image_width} x {image_height}")
print(f"Label names: {class_names}")
print(f"Label counts: {num_each}")

In [ ]:
#随机挑选九张图片进行可视化
plt.subplots(3, 3, figsize=(8, 8))
for i, k in enumerate(np.random.randint(num_total, size=9)):
    im = PIL.Image.open(image_files_list[k])
    arr = np.array(im)
    plt.subplot(3, 3, i + 1)
    plt.xlabel(class_names[image_class[k]])
    plt.imshow(arr, cmap="gray", vmin=0, vmax=255)
plt.tight_layout()
plt.show()

In [ ]:
#划分数据集
val_frac = 0.1
test_frac = 0.1
length = len(image_files_list)
indices = np.arange(length)
np.random.shuffle(indices)

test_split = int(test_frac * length)
val_split = int(val_frac * length) + test_split
test_indices = indices[:test_split]
val_indices = indices[test_split:val_split]
train_indices = indices[val_split:]

train_x = [image_files_list[i] for i in train_indices]
train_y = [image_class[i] for i in train_indices]
val_x = [image_files_list[i] for i in val_indices]
val_y = [image_class[i] for i in val_indices]
test_x = [image_files_list[i] for i in test_indices]
test_y = [image_class[i] for i in test_indices]

print(f"Training count: {len(train_x)}, Validation count: " f"{len(val_x)}, Test count: {len(test_x)}")

In [ ]:
#数据增强，归一化，随机旋转，随机翻转，随机缩放。
train_transforms = Compose(
    [
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        ScaleIntensity(),
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        RandFlip(spatial_axis=0, prob=0.5),
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
    ]
)

val_transforms = Compose([LoadImage(image_only=True), EnsureChannelFirst(), ScaleIntensity()])

y_pred_trans = Compose([Activations(softmax=True)])
y_trans = Compose([AsDiscrete(to_onehot=num_class)])

In [ ]:
#构建dataloader
class MedNISTDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]


train_ds = MedNISTDataset(train_x, train_y, train_transforms)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=12)

val_ds = MedNISTDataset(val_x, val_y, val_transforms)
val_loader = DataLoader(val_ds, batch_size=16, num_workers=12)

test_ds = MedNISTDataset(test_x, test_y, val_transforms)
test_loader = DataLoader(test_ds, batch_size=16, num_workers=12)

In [ ]:
#构建模型
import torch
import torch.nn as nn
import timm
model = timm.create_model('efficientnetv2_s', pretrained=False)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, 2)

In [ ]:
#构建损失和优化器
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
# model = DenseNet121(spatial_dims=2, in_channels=3, out_channels=num_class).to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-5)
max_epochs = 200
val_interval = 1
auc_metric = ROCAUCMetric()

In [ ]:
#构建训练和验证过程
best_metric = -1
best_metric_epoch = -1
train_loss_values = []
train_acc_values =[]
train_auc_values=[]
val_loss_values=[]
val_acc_values=[]
val_auc_values = []

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    train_loss = 0
    step = 0
    train_y_pred = torch.tensor([], dtype=torch.float32, device='cpu')
    train_y = torch.tensor([], dtype=torch.long, device='cpu')
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        train_y_pred = torch.cat([train_y_pred, outputs.cpu()], dim=0)
        train_y = torch.cat([train_y, labels.cpu()], dim=0)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        print(f"{step}/{len(train_ds) // train_loader.batch_size}, " f"train_loss: {loss.item():.4f}")
        epoch_len = len(train_ds) // train_loader.batch_size
    y_onehot = [y_trans(i) for i in decollate_batch(train_y, detach=False)]
    y_pred_act = [y_pred_trans(i) for i in decollate_batch(train_y_pred)]
    auc_metric(y_pred_act, y_onehot)
    train_auc = auc_metric.aggregate()
    auc_metric.reset()
    train_auc_values.append(train_auc)
    del y_pred_act, y_onehot
    acc_value = torch.eq(train_y_pred.argmax(dim=1), train_y)
    train_acc = acc_value.sum().item() / len(acc_value)
    train_acc_values.append(train_acc)
    train_loss /= step
    train_loss_values.append(train_loss)
    print(f"epoch {epoch + 1} average loss: {train_loss:.4f}")
    

    if (epoch + 1) % val_interval == 0:
        model.eval()
        val_loss=0
        val_step=0
        with torch.no_grad():
            val_y_pred = torch.tensor([], dtype=torch.float32, device=device)
            val_y = torch.tensor([], dtype=torch.long, device=device)
            for val_data in val_loader:
                val_step += 1
                val_images, val_labels = (
                    val_data[0].to(device),
                    val_data[1].to(device),
                )
                val_outputs=model(val_images)
                val_y_pred = torch.cat([val_y_pred, val_outputs], dim=0)
                val_y = torch.cat([val_y, val_labels], dim=0)
                loss = loss_function(val_outputs, val_labels)
                val_loss += loss.item()
            val_loss /= val_step
            val_loss_values.append(val_loss)
            y_onehot = [y_trans(i) for i in decollate_batch(val_y, detach=False)]
            y_pred_act = [y_pred_trans(i) for i in decollate_batch(val_y_pred)]
            auc_metric(y_pred_act, y_onehot)
            val_auc = auc_metric.aggregate()
            auc_metric.reset()
            del y_pred_act, y_onehot
            val_auc_values.append(val_auc)
            acc_value = torch.eq(val_y_pred.argmax(dim=1), val_y)
            val_acc = acc_value.sum().item() / len(acc_value)
            val_acc_values.append(val_acc)
            if val_auc > best_metric:
                best_metric = val_auc
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), os.path.join(root_dir, "efficientv2_l_best_metric_model.pth"))
                print("saved new best metric model")
    print(
        f"current epoch: {epoch + 1}"
        f" train AUC: {train_auc:.4f} " 
        f" train loss: {train_loss:.4f}"
        f" train accuracy: {train_acc:.4f}"
        f" val AUC: {val_auc:.4f} " 
        f" val loss: {val_loss:.4f}"
        f" val accuracy: {val_acc:.4f}"
        f" best AUC: {best_metric:.4f}"
        f" at epoch: {best_metric_epoch}"
    )

print(f"train completed, best_metric: {best_metric:.4f} " f"at epoch: {best_metric_epoch}")

In [ ]:
#对loss,acc,auc进行可视化
plt.figure(figsize=(12, 6))
plt.subplot(1,3,1)
plt.plot(train_loss_values, label='train loss')
plt.plot(val_loss_values, label='val loss')
plt.legend(loc='upper left')
plt.xlabel("epoch")
plt.title(" loss")
plt.subplot(1,3,2)
plt.plot(train_acc_values, label='train acc')
plt.plot(val_acc_values, label='val acc')
plt.legend(loc='upper left')
plt.xlabel("epoch")
plt.title(" acc")
plt.subplot(1,3,3)
plt.plot(train_auc_values, label='train auc')
plt.plot(val_auc_values, label='val auc')
plt.legend(loc='upper left')
plt.xlabel("epoch")
plt.title(" auc")
plt.show()

In [ ]:
#对测试集进行测试
model.load_state_dict(torch.load(os.path.join(root_dir, "efficientv2_s_best_metric_model.pth"), map_location='cpu'))
model.to(device)
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    y_pred = torch.tensor([], dtype=torch.float32, device=device)
    y = torch.tensor([], dtype=torch.long, device=device)
    for test_data in test_loader:
        test_images, test_labels = (
            test_data[0].to(device),
            test_data[1].to(device),
        )
        pred = model(test_images)
        y_pred = torch.cat([y_pred, pred], dim=0)
        y = torch.cat([y, test_labels], dim=0)
        # for i in range(len(pred)):
        #     y_true.append(test_labels[i].item())
        #     y_pred.append(pred[i].item())
    y_onehot = [y_trans(i) for i in decollate_batch(y, detach=False)]
    y_pred_act = [y_pred_trans(i) for i in decollate_batch(y_pred)]
    auc_metric(y_pred_act, y_onehot)
    result = auc_metric.aggregate()
    auc_metric.reset()
    del y_pred_act, y_onehot
    acc_value = torch.eq(y_pred.argmax(dim=1), y)
    print(len(acc_value))
    test_acc = acc_value.sum().item() / len(acc_value)
    print('test acc:',test_acc)
    print('test auc:',result)